In [ ]:
"""
Takes [hml_filename].hml and [txt_filename].txt
and converts them into csv formats

The code will try to read the GPU and app name from the log file(s)

    NO .txt -> NO app name
    NO .hml -> NO GPU name

Overwrite the names if needed

You need at least one of the two files
The two files should be located in the same folder
Errors -> Blame Copilot
"""

import csv
from pathlib import Path
encoder = "cp1252"

# Edit the path and filenames
log_path = "C:/Users/wafffle/Downloads/Logs/"
hml_filename = "HardwareMonitoring"
txt_filename = "Benchmark"

# Overwrite the columns in csv if not empty
# Edit app_name if the .exe file name is as clear as mud to the reader
gpu_name = ""
app_name = "Wuthering Waves"


In [32]:
# Path to the HML file
hml_file = Path(log_path + hml_filename).with_suffix('.hml')
csv_file = Path(log_path + hml_filename).with_suffix('.csv')

# Path to the benchmark file
benchmark_file = Path(log_path + txt_filename).with_suffix('.txt')
benchmark_csv = Path(log_path + txt_filename).with_suffix('.csv')

# Check if files exist
if not hml_file.exists():
    print(f"{hml_file} not found, cannot convert to CSV")
if not benchmark_file.exists():
    print(f"{benchmark_file} not found, cannot convert to CSV")

# Read the HML file and write to CSV
headers = []
units = []
data_rows = []

# Get application from benchmark
if benchmark_file.exists():
    with open(benchmark_file, 'r', encoding=encoder) as f:
        for line in f:
            if 'benchmark completed' in line:
                parts = line.split(',')
                time_app = parts[1].strip()
                time, app_part = time_app.split(' ', 1)
                if not app_name:
                    app_name = app_part.split(' ')[0]
                break

if hml_file.exists():
    with open(hml_file, 'r', newline='', encoding=encoder) as infile:
        reader = csv.reader(infile)
        for row in reader:
            if row and row[0] == '01':
                # Extract GPU name (first GPU if multiple)
                if not gpu_name:
                    gpu_name = row[2].split(',')[0].strip()
            elif row and row[0] == '02':
                headers = row[2:]  # Skip type and timestamp
            elif row and row[0] == '03':
                units.append(row[3].strip())  # Unit is in column 3
            elif row and row[0] == '80':
                data_rows.append(row[1:])  # Timestamp and values

# Combine headers with units
combined_headers = ["GPU", "Application", "Timestamp"] + [f"{h.strip()} ({u})" if u else h.strip() for h, u in zip(headers, units)]

# Write to CSV if data exists
if data_rows:
    with open(csv_file, 'w', newline='', encoding=encoder) as outfile:
        writer = csv.writer(outfile)
        writer.writerow(combined_headers)
        for row in data_rows:
            writer.writerow([gpu_name, app_name] + row)
    print(f"Converted {hml_file} to {csv_file}")
else:
    print(f"No data to convert from {hml_file}")

# Now process benchmark.txt
benchmark_data = []

if benchmark_file.exists():
    with open(benchmark_file, 'r', encoding=encoder) as f:
        lines = f.readlines()

    i = 0
    while i < len(lines):
        line = lines[i].strip()
        if 'benchmark completed' in line:
            # Parse timestamp and app
            parts = line.split(',')
            date = parts[0].strip()
            time_app = parts[1].strip()
            time, app_parsed = time_app.split(' ', 1)
            app_parsed = app_parsed.split(' ')[0]  # Client-Win64-Shipping.exe
            if app_name:
                app_parsed = app_name
            timestamp = f"{date} {time}"
            # Next 5 lines are metrics
            metrics = []
            for j in range(1, 6):
                if i + j < len(lines):
                    metric_line = lines[i + j].strip()
                    if ':' in metric_line:
                        value = metric_line.split(':')[1].strip().split(' ')[0]
                        metrics.append(value)
            if len(metrics) == 5:
                benchmark_data.append([gpu_name, app_parsed, timestamp] + metrics)
            i += 6  # Skip the 5 metric lines
        else:
            i += 1

# Write benchmark CSV if data exists
if benchmark_data:
    benchmark_headers = ['GPU', 'Application', 'Timestamp', 'Average Framerate', 'Minimum Framerate', 'Maximum Framerate', '1% Low Framerate', '0.1% Low Framerate']
    with open(benchmark_csv, 'w', newline='', encoding=encoder) as outfile:
        writer = csv.writer(outfile)
        writer.writerow(benchmark_headers)
        writer.writerows(benchmark_data)
    print(f"Converted {benchmark_file} to {benchmark_csv}")
else:
    print(f"No data to convert from {benchmark_file}")

Converted C:\Users\wafffle\Downloads\Logs\HardwareMonitoring.hml to C:\Users\wafffle\Downloads\Logs\HardwareMonitoring.csv
Converted C:\Users\wafffle\Downloads\Logs\Benchmark.txt to C:\Users\wafffle\Downloads\Logs\Benchmark.csv
